In [2]:
import google.generativeai as genai
import pandas as pd
import time
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

C:\Users\fabio\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
manual_df = pd.read_csv("data/manual_annotated_dataset/manual_data.csv")
cleaned_df = pd.read_csv("data/cleaned_dataset/merged.tsv", sep=",", encoding="utf-8")

In [16]:
merged_df = manual_df.merge(cleaned_df, how='left', left_on='id', right_on="ID")

In [17]:
#Rimuovo i testi usati nel few shot prompt
prompt_text = ["ParlaMint-IT_2014-03-26-LEG17-Senato-sed-217.u153", "ParlaMint-IT_2013-06-25-LEG17-Senato-sed-50.u53", "ParlaMint-IT_2018-06-05-LEG18-Senato-sed-9.u62","ParlaMint-IT_2021-06-09-LEG18-Senato-sed-334.u46"]
ground_df = merged_df.drop(merged_df[merged_df['id'].isin(prompt_text)].index)

In [18]:
#Estraggo i testi da dare in input a gemini
texts_to_classify = []
for id in ground_df["id"]:
    texts_to_classify.append((id, ground_df[ground_df["id"] == id].text.iloc[0]))

### GEN AI

In [ ]:
API_KEY = ""
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [20]:
def classify_record(text_id, text, prompt, model):
    filled_prompt = prompt.replace('[$$$]',text)
    res = model.generate_content(filled_prompt)
    time.sleep(4)
    return str(text_id + "," +res.text)

In [37]:
#CALCOLO CLASSIFICAZIONI GEMINI
def calcola_classificazioni(texts_to_classify, prompt):
    i = 0
    gemini_classes = []
    for tupla in texts_to_classify:
        res = classify_record(tupla[0], tupla[1], prompt, model)
        gemini_classes.append(res)
        i+=1
        print(i, end = " ")
        
    classification_list = []
    for stringa in gemini_classes:
        stringa = stringa.strip("\n")
        lista = stringa.split(",")
        lista = [lista[0]] + [int(x) for x in lista[1:]]
        classification_list.append(lista)

    classification_df = pd.DataFrame(classification_list, columns = ['id', 'manichean', 'indivisible', 'peoplecentrism', 'antielitism'])
    return classification_df

In [38]:
#CALCOLO CLASSIFICAZIONI GEMINI - CON PROB
def calcola_classificazioni_prob(texts_to_classify, prompt):
    i = 0
    gemini_classes = []
    for tupla in texts_to_classify:
        res = classify_record(tupla[0], tupla[1], prompt, model)
        gemini_classes.append(res)
        i+=1
        print(i, end = " ")
        
    classification_list = []
    for stringa in gemini_classes:
        stringa = stringa.strip("\n")
        lista = stringa.split(",")
        lista = [lista[0]] + [int(x) for x in lista[1:]]
        classification_list.append(lista)

    classification_df = pd.DataFrame(classification_list, columns = ['id', 'manichean', 'indivisible', 'peoplecentrism', 'antielitism', 'prob'])
    return classification_df

In [22]:
def calcola_prestazioni(ground_df, classification_df):
    dimensions = ['manichean', 'indivisible', 'peoplecentrism', 'antielitism']
    for dim in dimensions:
        print("ANALISI: ", dim)
        print("SPEARMAN: ", stats.spearmanr(ground_df[dim], classification_df[dim]))
        print("MSE: ", mean_squared_error(ground_df[dim], classification_df[dim]))
        print("MAE: ", mean_absolute_error(ground_df[dim], classification_df[dim]))
        print(classification_report(ground_df[dim], classification_df[dim]))
        print("\n\n")

### Few Shot

In [43]:
with open('genai/prompt.txt', 'r', encoding="utf8") as f:
    prompt = f.read()

In [44]:
classification_df = pd.read_csv("genai/50_texts_classifications.csv", sep=",", encoding="utf-8")
#classification_df = calcola_classificazioni(texts_to_classify, prompt)

In [45]:
#classification_df.to_csv("genai/50_texts_classifications.csv", index=False)

In [46]:
calcola_prestazioni(ground_df, classification_df)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.4798490862098994, pvalue=0.0004215637593474558)
MSE:  0.72
MAE:  0.68
              precision    recall  f1-score   support

           0       0.48      0.62      0.54        16
           1       0.38      0.10      0.16        30
           2       0.19      1.00      0.32         4

    accuracy                           0.34        50
   macro avg       0.35      0.58      0.34        50
weighted avg       0.39      0.34      0.29        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.34079272776582015, pvalue=0.015443709024348141)
MSE:  0.26
MAE:  0.22
              precision    recall  f1-score   support

           0       0.80      0.97      0.88        38
           1       0.75      0.27      0.40        11
           2       0.00      0.00      0.00         1

    accuracy                           0.80        50
   macro avg       0.52      0.42      0.43        50
weighted avg       0.78 

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Zero Shot

In [1]:
with open('genai/zero-shot-prompt.txt', 'r', encoding="utf8") as f:
    zero_shot_prompt = f.read()

In [47]:
zero_shot_classification = pd.read_csv("genai/zero-shot_50_texts_classifications.csv", sep=",", encoding="utf-8")
#zero_shot_classification = calcola_classificazioni(texts_to_classify, zero_shot_prompt)

In [48]:
#zero_shot_classification.to_csv("genai/zero-shot_50_texts_classifications.csv", index=False)

In [49]:
calcola_prestazioni(ground_df, zero_shot_classification)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.47194179129308417, pvalue=0.0005401452787881557)
MSE:  0.48
MAE:  0.48
              precision    recall  f1-score   support

           0       0.48      0.75      0.59        16
           1       0.69      0.37      0.48        30
           2       0.33      0.75      0.46         4

    accuracy                           0.52        50
   macro avg       0.50      0.62      0.51        50
weighted avg       0.59      0.52      0.51        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.46561492289988066, pvalue=0.0006558392124161011)
MSE:  0.2
MAE:  0.2
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        38
           1       0.57      0.36      0.44        11
           2       0.00      0.00      0.00         1

    accuracy                           0.80        50
   macro avg       0.47      0.44      0.44        50
weighted avg       0.76 

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Binary transformation

In [58]:
bin_ground_df = ground_df[['id','manichean', 'indivisible', 'peoplecentrism', 'antielitism']].replace(2, 1)
bin_classification_df = classification_df.replace(2, 1)
bin_zero_classification_df = zero_shot_classification.replace(2, 1)

In [51]:
calcola_prestazioni(bin_ground_df, bin_classification_df)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.28492840823101134, pvalue=0.0449016983729226)
MSE:  0.34
MAE:  0.34
              precision    recall  f1-score   support

           0       0.48      0.62      0.54        16
           1       0.79      0.68      0.73        34

    accuracy                           0.66        50
   macro avg       0.63      0.65      0.64        50
weighted avg       0.69      0.66      0.67        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.3521347126246435, pvalue=0.012145513123501699)
MSE:  0.2
MAE:  0.2
              precision    recall  f1-score   support

           0       0.80      0.97      0.88        38
           1       0.75      0.25      0.38        12

    accuracy                           0.80        50
   macro avg       0.78      0.61      0.63        50
weighted avg       0.79      0.80      0.76        50




ANALISI:  peoplecentrism
SPEARMAN:  SignificanceResult(statistic=0.552328574607

In [52]:
calcola_prestazioni(bin_ground_df, bin_zero_classification_df)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.34299717028501775, pvalue=0.014748787378676192)
MSE:  0.34
MAE:  0.34
              precision    recall  f1-score   support

           0       0.48      0.75      0.59        16
           1       0.84      0.62      0.71        34

    accuracy                           0.66        50
   macro avg       0.66      0.68      0.65        50
weighted avg       0.72      0.66      0.67        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.44806644233734894, pvalue=0.0011022839997438436)
MSE:  0.18
MAE:  0.18
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        38
           1       0.71      0.42      0.53        12

    accuracy                           0.82        50
   macro avg       0.78      0.68      0.71        50
weighted avg       0.81      0.82      0.80        50




ANALISI:  peoplecentrism
SPEARMAN:  SignificanceResult(statistic=0.356348

### Zero-shot 2.0

In [42]:
with open('genai/zero-shot_prompt_2.0.txt', 'r', encoding="utf8") as f:
    zero_shot_prompt_2 = f.read()

In [48]:
zero_shot_classification = pd.read_csv("genai/zero-shot_50_texts_classifications_2.0.csv", sep=",", encoding="utf-8")
#zero_shot_classification_2 = calcola_classificazioni_prob(texts_to_classify, zero_shot_prompt_2)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

In [64]:
ground_df.shape

(50, 20)

In [71]:
ground_df[ground_df["prob"]>7].shape

(40, 20)

In [72]:
zero_shot_classification_2[zero_shot_classification_2["prob"]>8].shape

(40, 6)

In [79]:
zero_shot_classification_2.to_csv("genai/zero-shot_50_texts_classifications_2.0.csv", index=False)

In [76]:
calcola_prestazioni(ground_df[ground_df["prob"]>7], zero_shot_classification_2[zero_shot_classification_2["prob"]>7])

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.011519450637589478, pvalue=0.9437579588512655)
MSE:  0.925
MAE:  0.725
              precision    recall  f1-score   support

           0       0.38      0.43      0.40        14
           1       0.56      0.39      0.46        23
           2       0.00      0.00      0.00         3

    accuracy                           0.38        40
   macro avg       0.31      0.27      0.29        40
weighted avg       0.45      0.38      0.41        40




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=-0.24069390083941855, pvalue=0.13462654509813718)
MSE:  0.625
MAE:  0.575
              precision    recall  f1-score   support

           0       0.67      0.53      0.59        30
           1       0.12      0.22      0.16         9
           2       0.00      0.00      0.00         1

    accuracy                           0.45        40
   macro avg       0.26      0.25      0.25        40
weighted avg       0.

C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Binary directly

In [84]:
bin_ground_df = ground_df[['id','manichean', 'indivisible', 'peoplecentrism', 'antielitism']].replace(2, 1)

In [82]:
with open('genai/binary_few_prompt.txt', 'r', encoding="utf8") as f:
    binary_few_prompt = f.read()

In [3]:
binary_few_prompt_classification = pd.read_csv("genai/binary_few_classifications.csv", sep=",", encoding="utf-8")
#binary_few_prompt_classification = calcola_classificazioni(texts_to_classify, binary_few_prompt)

In [88]:
#binary_few_prompt_classification.to_csv("genai/binary_few_classifications.csv", index=False)

In [91]:
calcola_prestazioni(bin_ground_df, binary_few_prompt_classification)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.4353221193479409, pvalue=0.0015802285746497938)
MSE:  0.32
MAE:  0.32
              precision    recall  f1-score   support

           0       0.50      0.88      0.64        16
           1       0.91      0.59      0.71        34

    accuracy                           0.68        50
   macro avg       0.70      0.73      0.68        50
weighted avg       0.78      0.68      0.69        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.44806644233734894, pvalue=0.0011022839997438436)
MSE:  0.18
MAE:  0.18
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        38
           1       0.71      0.42      0.53        12

    accuracy                           0.82        50
   macro avg       0.78      0.68      0.71        50
weighted avg       0.81      0.82      0.80        50




ANALISI:  peoplecentrism
SPEARMAN:  SignificanceResult(statistic=0.534522